In [ ]:

import os
import wandb
import requests
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import requests_cache
import openmeteo_requests
from sklearn.metrics import mean_squared_error
from retry_requests import retry
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")

# set repeatability
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)     # Set a random seed for CUDA operations.
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    
    # Ensure deterministic behavior for CUDA operations (note: If you are not concerned with reportable reproducibility, set deterministic to false, and benchmark to true - as it can choose faster algorithms).
    torch.backends.cudnn.deterministic = True  # Set cuDNN to deterministic mode - it will now only select algorithms that are known to be deterministic.
    torch.backends.cudnn.benchmark = False  # Disable cuDNN benchmarking - it may select the best algorithms for the hardware, but it doesn't guarantee deterministic results.   

In [ ]:
# Setup Open-Meteo API client with cache and retry
def fetch_weather_data():
    cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    # Fetch weather data
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": 27.9475,
        "longitude": -82.4584,
        "start_date": "1990-03-14",
        "end_date": "2025-03-28",
        "hourly": [
            "temperature_2m", "precipitation", "pressure_msl",
            "surface_pressure", "relative_humidity_2m", "wind_speed_10m"
        ]
    }
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]  # Process first location

    # Extract hourly weather data
    hourly = response.Hourly()
    hourly_data = {
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ),
        "temperature": hourly.Variables(0).ValuesAsNumpy(),
        "humidity": hourly.Variables(4).ValuesAsNumpy(),
        "wind_speed": hourly.Variables(5).ValuesAsNumpy(),
        "precipitation": hourly.Variables(1).ValuesAsNumpy(),
        "pressure_msl": hourly.Variables(2).ValuesAsNumpy(),
        "surface_pressure": hourly.Variables(3).ValuesAsNumpy(),
    }

    return pd.DataFrame(data=hourly_data)


In [ ]:
def time_series_split(df, train_ratio=0.7, val_ratio=0.15):
    """Splits the data into train, validation, and test sets."""
    n = len(df)
    train_end = int(n * train_ratio)
    val_end = train_end + int(n * val_ratio)

    train_data = df.iloc[:train_end]
    val_data = df.iloc[train_end:val_end]
    test_data = df.iloc[val_end:]

    return train_data, val_data, test_data

In [ ]:
def create_sequences(data,seq_len):
        sequences, targets = [], []
        for i in range(len(data) - seq_len):
            sequences.append(data.iloc[i:i+seq_len].values)
            targets.append(data.iloc[i+seq_len]["temperature"])  # Predict temperature
        return np.array(sequences), np.array(targets)

In [ ]:
def process_data(df, seq_len=4):
    """Processes data for training."""
    df = df.drop(columns=["date"])
    print(df.head)
    # Split data
    train_data, val_data, test_data = time_series_split(df)

    # Normalize each split separately using training set statistics
    scaler = MinMaxScaler()
    train_data = pd.DataFrame(scaler.fit_transform(train_data), columns=df.columns, index=train_data.index)
    val_data = pd.DataFrame(scaler.transform(val_data), columns=df.columns, index=val_data.index)
    test_data = pd.DataFrame(scaler.transform(test_data), columns=df.columns, index=test_data.index)


    X_train, y_train = create_sequences(train_data, seq_len)
    X_val, y_val = create_sequences(val_data,seq_len)
    X_test, y_test = create_sequences(test_data,seq_len)

    return X_train, y_train, X_val, y_val, X_test, y_test, scaler


In [ ]:
wandb_api_key = "dc94a6f2fb35e68dd0ce5b185a38960232124958"
if wandb_api_key:
    wandb.login(key=wandb_api_key)
else:
    raise ValueError("WANDB_API_KEY environment variable not set!")

# Load the API key from the environment variable
wandb.init(project="Assignment6", entity="usf-magma", config={
    "learning_rate": 0.0002,
    "dropout": 0.4,
    "batch_size": 64,
    "epochs": 10,
    "momentum": (0.9, 0.92),
    "weight_decay": 1e-5,
    "optimizer": "Adam",
    "criterion": "MSE",  # Changed to MAE
    "input_size": 1,
    "hidden_size": 64
})

config = wandb.config

In [ ]:
def prepare_dataloader(X, y, batch_size):
    """Create a DataLoader from given features and targets."""
    dataset = TensorDataset(torch.tensor(X, dtype=torch.float32), 
                            torch.tensor(y, dtype=torch.float32).unsqueeze(1))
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, save_dir, device):
    """Train the model, log metrics, and save the best model based on validation loss."""

    model.to(device)  # Move model to device (CPU/GPU)
    train_losses, val_losses = [], []
    train_mae, val_mae = [], []
    best_loss = float("inf")  # Track best validation loss
    best_mae = float("inf")   # Track lowest validation MAE

    for epoch in range(num_epochs):
        model.train()
        running_loss, running_mae = 0.0, 0.0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # Move batch to GPU/CPU
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_mae += nn.functional.l1_loss(y_pred, y_batch).item()

        train_loss_epoch = running_loss / len(train_loader)
        train_mae_epoch = running_mae / len(train_loader)
        train_losses.append(train_loss_epoch)
        train_mae.append(train_mae_epoch)

        # Validation
        model.eval()
        val_loss, val_mae_epoch = 0.0, 0.0
        with torch.no_grad():
            for X_val_batch, y_val_batch in val_loader:
                X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)  # Move to GPU/CPU
                y_val_pred = model(X_val_batch)
                val_loss += criterion(y_val_pred, y_val_batch).item()
                val_mae_epoch += nn.functional.l1_loss(y_val_pred, y_val_batch).item()

        val_loss_epoch = val_loss / len(val_loader)
        val_mae_epoch = val_mae_epoch / len(val_loader)
        val_losses.append(val_loss_epoch)
        val_mae.append(val_mae_epoch)

        print(f"Epoch {epoch+1}/{num_epochs} -> Train Loss: {train_loss_epoch:.4f}, Val Loss: {val_loss_epoch:.4f}, Train MAE: {train_mae_epoch:.4f}, Val MAE: {val_mae_epoch:.4f}")

        # Log to wandb
        wandb.log({
            "Train Loss": train_loss_epoch,
            "Validation Loss": val_loss_epoch,
            "Train MAE": train_mae_epoch,
            "Validation MAE": val_mae_epoch
        })

        # Save the best model based on validation loss
        if val_loss_epoch < best_loss:
            best_loss = val_loss_epoch
            best_mae = val_mae_epoch  # Track best MAE
            model_save_path = os.path.join(save_dir, f"dual_rnn_best_epoch_{epoch+1}.pth")
            torch.save(model.state_dict(), model_save_path)
            wandb.save(model_save_path)
            print(f"Model saved: {model_save_path}")

            # Update wandb summary with best values
            wandb.run.summary["Best Validation Loss"] = best_loss
            wandb.run.summary["Best Validation MAE"] = best_mae

    # Plot Train Loss vs Validation Loss
    plt.figure(figsize=(10,5))
    plt.plot(train_losses, label="Train Loss")
    plt.plot(val_losses, label="Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Train Loss vs Validation Loss")
    plt.legend()
    plt.show()

    # Plot Train MAE vs Validation MAE
    plt.figure(figsize=(10,5))
    plt.plot(train_mae, label="Train MAE")
    plt.plot(val_mae, label="Validation MAE")
    plt.xlabel("Epochs")
    plt.ylabel("MAE")
    plt.title("Train MAE vs Validation MAE")
    plt.legend()
    plt.show()

    return model


In [ ]:

def evaluate_model(model, test_loader, scaler, device):
    """Evaluate the model on the test set and plot Actual vs Prediction."""
    model.to(device)  # Ensure model is on the correct device
    model.eval()

    actuals, predictions = [], []

    with torch.no_grad():
        for X_test_batch, y_test_batch in test_loader:
            X_test_batch, y_test_batch = X_test_batch.to(device), y_test_batch.to(device)  # Move to GPU/CPU
            y_test_pred = model(X_test_batch)

            # Convert tensors to NumPy
            y_test_pred_np = y_test_pred.cpu().numpy().reshape(-1, 1)
            y_test_actual_np = y_test_batch.cpu().numpy().reshape(-1, 1)

            # Apply inverse transform correctly
            # Create a matrix with the predicted values in the first column and zeros for others
            y_test_pred_full = np.zeros((y_test_pred_np.shape[0], scaler.scale_.shape[0]))  # Match feature dimension
            y_test_pred_full[:, 0] = y_test_pred_np.flatten()  # Fill temperature column with predictions

            y_test_actual_full = np.zeros((y_test_actual_np.shape[0], scaler.scale_.shape[0]))  # Same for actuals
            y_test_actual_full[:, 0] = y_test_actual_np.flatten()  # Fill temperature column with actuals

            # Apply inverse transform for both predicted and actual values
            y_test_pred_original = scaler.inverse_transform(y_test_pred_full)[:, 0]  # Get only temperature
            y_test_actual_original = scaler.inverse_transform(y_test_actual_full)[:, 0]  # Get only temperature

            actuals.extend(y_test_actual_original.tolist())
            predictions.extend(y_test_pred_original.tolist())

    # Compute final MSE
    test_mse = mean_squared_error(actuals, predictions)
    print(f"Test MSE: {test_mse:.4f}")

    # Print first 10 actual and predicted values
    print("First 10 Actual Values: ", actuals[:10])
    print("First 10 Predicted Values: ", predictions[:10])

    # Plot Actual vs Predicted Values
    plt.figure(figsize=(10, 5))
    plt.scatter(actuals, predictions, alpha=0.5, label="Predicted vs Actual")
    plt.plot([min(actuals), max(actuals)], [min(actuals), max(actuals)], color='red', linestyle='--', label="Perfect Prediction")
    plt.xlabel("Actual Temperature")
    plt.ylabel("Predicted Temperature")
    plt.title("Actual vs Predicted Temperature")
    plt.legend()
    plt.show()

    return test_mse

In [ ]:
class MultiHeadRNN(nn.Module):
    def __init__(self):
        super(MultiHeadRNN, self).__init__()
        self.rnn_temp = nn.RNN(input_size=1, hidden_size=config.hidden_size, num_layers=1, batch_first=True)
        self.rnn_hum = nn.RNN(input_size=1, hidden_size=config.hidden_size, num_layers=1, batch_first=True)
        self.rnn_wind = nn.RNN(input_size=1, hidden_size=config.hidden_size, num_layers=1, batch_first=True)
        self.rnn_precip = nn.RNN(input_size=1, hidden_size=config.hidden_size, num_layers=1, batch_first=True)
        self.rnn_pressure_msl = nn.RNN(input_size=1, hidden_size=config.hidden_size, num_layers=1, batch_first=True)
        self.rnn_surface_pressure = nn.RNN(input_size=1, hidden_size=config.hidden_size, num_layers=1, batch_first=True)
        
        self.fc1 = nn.Linear(config.hidden_size * 6, config.hidden_size)
        self.fc2 = nn.Linear(config.hidden_size, 1)
        self.dropout = nn.Dropout(config.dropout)
    
    def forward(self, x):
        x_temp = x[:, :, 0].unsqueeze(-1)  # Extract temperature
        x_hum = x[:, :, 1].unsqueeze(-1)   # Extract humidity
        x_wind = x[:, :, 2].unsqueeze(-1)  # Extract wind speed
        x_precip = x[:, :, 3].unsqueeze(-1)  # Extract precipitation
        x_pressure_msl = x[:, :, 4].unsqueeze(-1)  # Extract mean sea level pressure
        x_surface_pressure = x[:, :, 5].unsqueeze(-1)  # Extract surface pressure
        
        out_temp, _ = self.rnn_temp(x_temp)
        out_hum, _ = self.rnn_hum(x_hum)
        out_wind, _ = self.rnn_wind(x_wind)
        out_precip, _ = self.rnn_precip(x_precip)
        out_pressure_msl, _ = self.rnn_pressure_msl(x_pressure_msl)
        out_surface_pressure, _ = self.rnn_surface_pressure(x_surface_pressure)
        
        last_temp = out_temp[:, -1, :]
        last_hum = out_hum[:, -1, :]
        last_wind = out_wind[:, -1, :]
        last_precip = out_precip[:, -1, :]
        last_pressure_msl = out_pressure_msl[:, -1, :]
        last_surface_pressure = out_surface_pressure[:, -1, :]
        
        combined = torch.cat([last_temp, last_hum, last_wind, last_precip, last_pressure_msl, last_surface_pressure], dim=1)
        combined_1 = self.dropout(torch.tanh(self.fc1(combined)))
        prediction = self.fc2(combined_1)
        return prediction


In [ ]:
if __name__ == "__main__":

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Create a folder for saving models if it doesn't exist
    save_dir = "saved_models"
    os.makedirs(save_dir, exist_ok=True)

    data = fetch_weather_data()
    X_train, y_train, X_val, y_val, X_test, y_test, scaler = process_data(data)
    print(X_train.shape, y_train.shape)  # Debe ser (num_samples, SEQ_LEN, 3) y (num_samples,)
    print(X_train[:2])  # Muestra las primeras secuencias para verificar estructura
    print(y_train[:2])  # Muestra los valores esperados

    # Prepare data loaders
    train_loader = prepare_dataloader(X_train, y_train, config.batch_size)
    val_loader = prepare_dataloader(X_val, y_val, config.batch_size)
    test_loader = prepare_dataloader(X_test, y_test, config.batch_size)

    # Initialize model, loss function, and optimizer
    model = MultiHeadRNN().to(device)  # Move model to correct device
    criterion = nn.MSELoss()  # MSE Loss
    #criterion = nn.L1Loss()  # MAE Loss
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)

    # Train the model
    model = train_model(model, train_loader, val_loader, criterion, optimizer, config.epochs, save_dir, device)

    # Save final model
    final_model_path = os.path.join(save_dir, "multihead_rnn_final.pth")
    torch.save(model.state_dict(), final_model_path)
    wandb.save(final_model_path)
    print(f"✅ Final model saved: {final_model_path}")

    # Evaluate the model on test data
    evaluate_model(model, test_loader, scaler,device)

    # Finish wandb logging
    wandb.finish()